In [1]:
import pandas as pd
import numpy as np
import pickle
import gradio as gr

questions = [
    "1. Does/did your child ever point with his/her index finger to bring your attention to show the things that interest him/her? E.g. kite, plane flying in the sky, cow/dog on the road etc.",
    "2. For children aged 4 years or more: Does your child talk to you about things he/she likes or has achieved without being asked about them?",
    "3. Does your child usually prefer to play alone and gets irritated/moves away when his/her sibs or other kids try to play with him/her?",
    "4. Does your child play games involving turn taking or rule based with other children properly? E.g. Cricket, Hide and seek/I-spy, Ludo, Stapoo, Ring-a-ring roses etc.",
    "5. Does your child usually share his/her happiness with you or come to you for comfort when hurt or upset?",
    "6. For children aged 4 years or more: Does your child usually share your happiness or try to comfort you when you are upset/sad?",
    "7. Does your child initiate a conversation with you?",
    "8. For children aged 4 years or more: Can you have conversation with your child during which he/she not only answers your questions, but also adds something new to continue the conversation?",
    "1. Does your child usually enjoy being taken in the lap or hugged? For children aged 4 years or more: When your child was a baby/toddler, did he/she enjoy being taken in the lap or hugged?",
    "2. Does your child usually make eye contact with you or other people? E.g. While playing, asking for things, talking to you.",
    "3. Does your child usually use various gestures appropriately during social interactions? E.g. Namaste, Salaam, waving bye-bye, hello, touching feet etc. (At least sometimes spontaneously) (use appropriate example as required)",
    "4. Does your child usually show appropriate facial expressions according to the situation? E.g. being happy, sad, afraid etc.",
    "1. Does your child usually enjoy the company of other children?",
    "2. For children aged 4 years or more: Does your child have friends of his/her age (In school and neighborhood) with whom he/she loves to chat, share food or play together?",
    "3. For children aged 4 years or more: Does your child play mostly with children who are much older or much younger than him/her?",
    "1. Does your child usually repeat words or phrases regardless of meaning (in part or whole) that he/she has heard? E.g. If you say 'toffee' he will also say 'toffee' if you say 'come' he will also say 'come' and if you ask 'what is your name', he/she also says 'what is your name'.",
    "2. Does he/she incessantly repeat things/T.V serial dialogue regardless of meaning/context, whatever he/she has heard later on?",
    "3. For children aged 4 years or more: Does your child usually use 'I for me' and 'me for you' incorrectly? E.g., when you ask 'do you want milk?' he/she says 'yes, you want milk' or 'Rohit wants milk' (referring to himself).",
    "4. For children aged 4 years or more: During conversation does your child often speak 'out of context' or irrelevantly?",
    "5. For children aged 6 years or more: Does your child understand that somebody is making fun of him/her or can he/she understand jokes?",
    "6. Does your child keep on repeating any of the following, like: flapping hands, hand wringing, toe-walking, rocking or spinning, making unusual finger or hand movements near his/her face?",
    "7. Does your child have inappropriate fascination with movement? E.g. spinning wheels, opening and closing of doors, electric fan, running water and any other revolving object etc.",
    "1. Does your child unreasonably insist on doing things in a particular way and/or become upset if there is any change in the daily routine? E.g., Taking exactly the same route to the school or market, insisting on food being served in the same pattern or sequence etc.",
    "1. Does your child prefer to play with a particular part of a toy/object rather than the whole toy/object? E.g. wheels of a toy rather than the whole toy And/Or Persistent unusual preoccupation with inanimate objects? E.g. Toffee wrappers, threads, bits of papers, flowing water And/Or Persistent behavioural attributes? E.g. Liking particular sound/visual stimuli, any particular color or form of cloth",
    "1. Is your child indifferent to pain or temperature? Apparent indifference to pain or temperature.",
    "2. Does your child show excess reaction to specific sound or texture? Getting irritated with certain specific sounds or texture of certain clothes.",
    "3. Does your child have excessive smelling? Excessive smelling of hands or arms.",
    "4. Does your child have excessive touching of objects? Excessive touching objects in the room."
]

question_to_section = {
    0: "Section A1a", 1: "Section A1a", 2: "Section A1a", 3: "Section A1a",
    4: "Section A1a", 5: "Section A1a", 6: "Section A1a", 7: "Section A1a",
    8: "Section A1b", 9: "Section A1b", 10: "Section A1b", 11: "Section A1b",
    12: "Section A1c", 13: "Section A1c", 14: "Section A1c", 15: "Section A2a",
    16: "Section A2a", 17: "Section A2a", 18: "Section A2a", 19: "Section A2a",
    20: "Section A2a", 21: "Section A2a", 22: "Section A2b", 23: "Section A2c",
    24: "Section A2d", 25: "Section A2d", 26: "Section A2d", 27: "Section A2d"
}

css = """
.gradio-container {
    background-color: white;
    font-family: Arial, sans-serif;
}

h1, h2 {
    color: black;
    text-align: center;
}

h3 {
    color: #ff033e;
    text-align: center;
}

p {
    color: #666;
    text-align: center;
}

.gradio-radio label {
    margin-bottom: 10px;
}

.gradio-button-primary {
    background-color: #4CAF50;
    color: white;
}

.gradio-button-secondary {
    background-color: #f44336;
    color: white;
}

.gr-textbox {
    text-align: center;
}

.gr-output-box {
    display: flex;
    justify-content: center;
    align-items: center;
    height: 100%;
}
"""

def predict_autism(*responses):
    if any(response == '' or response is None for response in responses):
        return "Incomplete response"
    

    input_data = {f"Q{i+1}": responses[i] for i in range(len(responses))}
    input_df = pd.DataFrame([input_data])
    input_df.replace({'yes': 1, 'no': 0, 'not applicable': np.nan}, inplace=True)
    
    
    model_file_path = 'random_forest_model2.pkl'
    with open(model_file_path, 'rb') as model_file:
        model = pickle.load(model_file)
    
    input_df = input_df[model.feature_names_in_]
    prediction = model.predict(input_df)
    
    return 'Autism' if prediction[0] == 1 else 'No Autism'


inputs = [
    gr.Radio(choices=['yes', 'no', 'not applicable'] if i in [1, 5, 7, 13, 14, 17, 18, 19] else ['yes', 'no'], label=f"{question_to_section[i]}: {question}")
    for i, question in enumerate(questions)
]


with gr.Blocks(css=css) as demo:
    gr.Markdown("# Autism Prediction")
    gr.Markdown("## Answer the following questions to get a prediction on autism")
    gr.Markdown("### 1. Please answer all the questions.")
    gr.Markdown("### 2. Click Submit to get a prediction.")
    gr.Markdown("### 3. Click Clear to reset the form.")
    
    with gr.Column():
        for input_radio in inputs:
            input_radio.render()
    
    submit = gr.Button("Submit", variant="primary")
    clear = gr.Button("Clear", variant="secondary")
    output = gr.Textbox(label="Prediction", placeholder="Awaiting input...")
    
    submit.click(predict_autism, inputs, output)
    clear.click(lambda: [None]*28, None, inputs)


demo.launch()


c:\Users\nipun\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
